## League table
* Modify PLACE_SCORE, ATTENDANCE_SCORE, and TITLE as needed
* Then run the following cell:

In [2]:
import numpy as np
import copy
import os 
import shutil # To manually copy files if needed for archiving

#Scores for 1st to 8th place - add extra elements as needed but keep first element = 0
PLACE_SCORE=np.array([0,125,100,75,60,40,30,20,10]) #NB keep first element = 0
#Score for people finishing outside the place scores
ATTENDANCE_SCORE = 10 
#Displayed at the top of the table
TITLE = "League Table"
Results=None
players=[]
current_week=None
FILENAME="results.csv"

def s_score(place):
    if place >= 999:
        return 0
    if place < len(PLACE_SCORE):
        return  PLACE_SCORE[place]
    else:
        return ATTENDANCE_SCORE

score = np.vectorize(s_score)

class Player:
    def __init__(self,name,scores) -> None:
        self.name = name
        self.scores=copy.deepcopy(scores)

def read_results():
    res_names = np.genfromtxt(FILENAME, delimiter=",",dtype=str) #scores as string
    res_scores = np.genfromtxt(FILENAME, delimiter=",",dtype=int) #name as Na

    results = res_scores.astype(object) #blank will be False - i.e. -1
    players = []

    for row in range(len(results)):
        players.append(Player(res_names[row][0],results[row][1:]))
        results[row][0] = res_names[row][0]
    current_week = len(results[0])-1         

    

    for player in players:
        if len(player.scores) != len(players[0].scores):
            raise AssertionError(f"Player {player.name} has unequal no of scores")
    
    #Validation on a per week level
    res_scores_t = res_scores.T
    week_players=[]
    for _,week in enumerate(res_scores_t[1:]): #first row is names
        #print(week)
        week_players.append(week.max())

        order = week.argsort()
        #for _o in range (len(order)-1):
        for _o in range (1,len(order)):
            if week[order[_o]] != -1:
                if week[order[_o-1]] == -1:
                    if week[order[_o]] != 1:
                        raise AssertionError(f"Week {_+1} has incorrect set of scores No 1st place")
                else:
                    if (week[order[_o]] - week[order[_o-1]]) != 1:
                        raise AssertionError(f"Week {_+1} has incorrect set of scores {week[order[_o]]} {week[order[_o-1]]}")


    return players, current_week, week_players

def make_table():
    players, current_week, week_players = read_results()

    table = []
    scores=[] #solely used for sorting

    for player in players:
        #print(f"{player.name} {player.scores}")

        score = score_player(player.scores,week_players)
        adjusted_score = score - (0.00001 * np.sum(player.scores != -1)) # tie-break to favour player with fewer games
        scores.append(adjusted_score)
        wins = (player.scores == 1).sum()
        table.append ([player.name, (player.scores>-1).sum() , wins, score])

    order = np.argsort(np.array(scores))

    print(f"{TITLE} week {current_week}")
    print(f"      Player                  Games   Wins     Points")
    for _ in range(len(order)-1,-1,-1):
        print(f"{len(order)-_:2d}    {table[order[_]][0]:20} {table[order[_]][1]:6d}     {table[order[_]][2]:2d}       {table[order[_]][3]:4d}")


def score_player(p_r,week_players):

    player_results = np.where(p_r<1,999,p_r) #map -1 values
    
    raw_scores = score(player_results)

    return raw_scores.sum()


def add_player(name):
    #Add a new row to the results table with the correct number of commas
    
    players, current_week, week_players = read_results()
    for player in players:
        if player.name == name:
            raise AssertionError("player {name} already exists")

    new_line="\n"+name+","*(current_week)
    f=open(FILENAME,"a")
    f.write(new_line)
    f.close()
     

def add_week():
    #TODO: ARCHIVE the csv and add an extra column to the csv
    #OR should just use git for archiving
    players, current_week, week_players = read_results()
    backup_filename = FILENAME+".bak"
    if os.path.exists(FILENAME):
        try:
            os.remove(backup_filename)
        except OSError:
            pass
        new=[]
        os.rename(FILENAME,backup_filename)
        
        with open(backup_filename,"r") as f_bak:
            current = f_bak.readlines()
            
            for line in current:
                if line[-1] == "\n":
                    new.append(f"{line[:-1]},\n")
                else:
                    new.append(f"{line},")  
        with open(FILENAME,"w") as f:
            f.writelines(new)
    else:
        raise AssertionError(f"{FILENAME} does not exist!")



### To create the table:

In [12]:
make_table()

League Table week 1
      Player                  Games   Wins     Points
 1    Player One                1      1        125
 2    Player Two                1      0        100


### To add a new week and player

In [13]:
add_player("Player Three")
add_week()


League Table week 2
      Player                  Games   Wins     Points
 1    Player One                1      1        125
 2    Player Two                1      0        100
 3    Player Three              0      0          0


* Change the pre-existing player names as appropriate
* add in the places for the week at the end of the commas

In [14]:
make_table()


League Table week 2
      Player                  Games   Wins     Points
 1    Player One                1      1        125
 2    Player Two                1      0        100
 3    Player Three              0      0          0
